In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import json
import re
import requests

In [2]:
#네이버 검색 Open API 사용 요청시 얻게되는 정보를 입력합니다
naver_client_id = "iqhe2JFz47hsd6I781RV"
naver_client_secret = "VAcpRXviRN"

In [5]:

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
 
def searchByTitle(title):
    myurl = 'https://openapi.naver.com/v1/search/movie.json?display=100&query=' + quote(title)
    request = urllib.request.Request(myurl)
    request.add_header("X-Naver-Client-Id",naver_client_id)
    request.add_header("X-Naver-Client-Secret",naver_client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        d = json.loads(response_body.decode('utf-8'))
        if (len(d['items']) > 0):
            return d['items']
        else:
            return None
 
    else:
        print("Error Code:" + rescode)
    return
 
def findItemByInput(items):
    for index, item in enumerate(items):
        navertitle = cleanhtml(item['title'])
        naversubtitle = cleanhtml(item['subtitle'])
        naverpubdate = cleanhtml(item['pubDate'])
        naveractor = cleanhtml(item['actor'])
        naverlink = cleanhtml(item['link'])
        naveruserScore = cleanhtml(item['userRating'])
 
        navertitle1 = navertitle.replace(" ","")
        navertitle1 = navertitle1.replace("-", ",")
        navertitle1 = navertitle1.replace(":", ",")
 
        #기자 평론가 평점을 얻어 옵니다
        spScore = getSpecialScore(naverlink)
 
        #네이버가 다루는 영화 고유 ID를 얻어 옵니다다
        naverid = re.split("code=", naverlink)[1]
 
        # 영화의 타이틀 이미지를 표시합니다
        # if (item['image'] != None and "http" in item['image']):
        #    response = requests.get(item['image'])
        #    img = Image.open(BytesIO(response.content))
        #    img.show()
 
        print(index, navertitle, naversubtitle, naveruserScore, spScore)
    
    return

In [6]:
# 아래 spScore(전문가 평점)을 가져오는 방법은 API 호출이 아니라 크롤링인 듯.
# 네이버 영화 크롤링시 참고 가능할 듯.


def getInfoFromNaver(searchTitle):
    items = searchByTitle(searchTitle)
 
    if (items != None):
        findItemByInput(items)
    else:
        print("No result")
        
    return
 
def get_soup(url):
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    return soup
 
#기자 평론가 평점을 얻어 옵니다
def getSpecialScore(URL):
    soup = get_soup(URL)
    scorearea = soup.find_all('div', "spc_score_area")
    newsoup = BeautifulSoup(str(scorearea), 'lxml')
    score = newsoup.find_all('em')
    if (score and len(score) > 5):
        scoreis = score[1].text + score[2].text + score[3].text + score[4].text
        return float(scoreis)
    else:
        return 0.0
 
getInfoFromNaver(u"미션")

0 아톰 - 어 테이크오버 미션 ATOM - A TAKEOVER MISSION 0.00 0.0
1 미션 파서블(가제)  0.00 0.0
2 미션 임파서블: 폴아웃 Mission: Impossible - Fallout 9.12 7.56
3 라스트 미션 The Mule 8.81 7.2
4 레오나르도 다 빈치 Leo Da Vinci: Mission Mona Lisa 7.98 0.0
5 서브미션 Submission 0.00 0.0
6 루이스 &amp; 루카 - 미션 투 더 문 Louis &amp; Luca &amp;#8211; Mission to the Moon 0.00 0.0
7 스페셜 시큐리티 : 라스트 미션 Security 4.92 0.0
8 텐: 시크릿 미션 Ten: The Secret Mission 5.67 0.0
9 하이앤로우 더 무비 3 파이널 미션 High &amp; Low The Movie 3 Final Mission 0.00 0.0
10 미션: 러브 Mission: Love 0.00 0.0
11 솔라 임펄스, 디 임파서블 라운드 더 월드 미션 Solar Impulse, the Impossible Round the World Mission 0.00 0.0
12 서브미션 Submission 0.00 0.0
13 미션 카트만두: 더 어드벤쳐스 오브 넬리 &amp; 사이먼 Mission Kathmandu: The Adventures of Nelly &amp; Simon 0.00 0.0
14 미션 페이즈 바스크 Mission Pays Basque 0.00 0.0
15 엑스포 67 미션 임파서블 Expo 67 Mission Impossible 0.00 0.0
16 미션 이스탄불 Nick: Off Duty 4.78 4.5
17 미션 이스탄불5: 더 파이널 Fegefeuer 9.00 0.0
18 미션 이스탄불4: 익스트림 데이 Der Grosse Schmerz 4.00 0.0
19 로제타미션 Rosetta Mission 0.00 0.0
20 파일럿 스카이 미션 A